In [101]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import pickle
import pandas as pd
import numpy as np
from psm_causal_effects import psm_causal_effects

# read data
with open('data.dat') as f:
    data = pickle.load(f)
f.close()

# adding subject ids
for i in range(len(data)):
    data[i]['subject'] = pd.Series(i+np.zeros(data[i].shape[0]), index=data[i].index, dtype=int)

# concatenatig into a single dataframe
data = pd.concat(data, axis=0)
data = data.reset_index(drop=True)

# keeping only relevant variables
data = data[['subject','mood','quality','mood_prev']]

# removing nan rows
data = data.dropna()
data = data.reset_index(drop=True)

md = smf.mixedlm('mood ~ quality', data, groups=data['subject'])
mdf = md.fit() 
print(mdf.summary())


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: mood       
No. Observations: 9162    Method:             REML       
No. Groups:       207     Scale:              0.7702     
Min. group size:  4       Likelihood:         -12204.1991
Max. group size:  133     Converged:          Yes        
Mean group size:  44.3                                   
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       3.910    0.072 54.616 0.000  3.770  4.050
quality         0.254    0.007 35.889 0.000  0.240  0.268
Intercept RE    0.832    0.098                           



In [102]:
md = smf.mixedlm('quality ~ mood_prev', data, groups=data['subject'])
mdf = md.fit() 
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: quality    
No. Observations: 9162    Method:             REML       
No. Groups:       207     Scale:              1.6732     
Min. group size:  4       Likelihood:         -15703.8220
Max. group size:  133     Converged:          Yes        
Mean group size:  44.3                                   
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       3.202    0.103 31.029 0.000  3.000  3.405
mood_prev       0.252    0.014 17.393 0.000  0.223  0.280
Intercept RE    1.056    0.088                           



In [103]:
data

,subject,mood,quality,mood_prev
0,0,3.666667,1.0,1.000000
1,0,3.666667,1.0,3.666667
2,0,3.333333,1.0,3.666667
3,0,3.000000,2.0,2.333333
4,0,3.333333,2.0,3.000000
5,0,0.666667,1.0,3.333333
6,0,1.500000,2.0,0.666667
7,0,3.000000,0.0,1.500000
8,0,2.000000,1.0,3.000000
9,0,4.000000,2.0,2.000000
